In [5]:
(defun burner (c h o)
  (let* ((w (min (floor h 2) o))
         (d (min c (floor (- o w) 2)))
         (m (min (- c d) (floor (- h (* 2 w)) 4))))
    (values w d m)))


BURNER

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::BURNER in DEFUN


In [6]:
(burner 939 3 694)

1

346

0